# Exercises for Bayesian Statistics

The following script is the solution the exercises for the probability theory section (ie. the exercises 12-16) of the course 'Statistics for Data Science' (WS 19/20) by Prof. Ostwald. The exercise sheets can be found at the website https://www.ewi-psy.fu-berlin.de/einrichtungen/arbeitsbereiche/computational_cogni_neurosc/teaching/Statistics_for_Data_Science_19_20.html.

## Configuration

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# For reproducibility, fix seed.
np.random.seed(42)

## 12. Foundations and Conjugate Inference

### 1. Batch vs. Recursive Bayesian Estimation for Beta-Binomial Model

In [7]:
from scipy.special import beta
from scipy.stats import binom

Samples are taken from

$p(x, \theta) = p(x|\theta)p(\theta) = Bin(x;\theta,n)Beta(\theta;a,\beta)$

In [11]:
# 1. Specification for prior distribution.

In [12]:
# 2. Specification of data likelihood given the parameter.

In [13]:
# 3. Batch Bayesian estimation.

In [9]:
n = 10
binom_prob = 7
binom(10, binom_prob)

### 2. Bias and Consistency Properties of the Posterior Expected Value (Beta-Binomial Model)

### 3. Bias and Consistency Properties of the Posterior Expected Value (Gaussian-Gaussian Model)